import de la table adresse au format csv

In [ ]:
link  = "/content/drive/MyDrive/dataEngineer/adressTable.csv"

import de la bibliothèque panda et de l'api nominatim

In [ ]:
import pandas as pd
from geopy.geocoders import Nominatim
geocoder = Nominatim()

df_adress = pd.read_csv(link);
df_adress.head()

/usr/local/lib/python3.8/dist-packages/geopy/geocoders/osm.py:132: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  warnings.warn(


,address_id,address,city,postal_code
0,1,318 CHE DE ROUMAGOUA,LA CIOTAT,13600
1,2,19 RUE DES DAMES,SAINTE SAVINE,10300
2,3,22 RUE AMIRAL GUEPRATTE,LE CONQUET,29217
3,4,6 BD DES ETINES,LE COTEAU,42120
4,5,35 AV DU 159EME RIA,BRIANCON,5100


In [ ]:
df_adress.isnull().sum()

address_id     0
address        0
city           0
postal_code    0
dtype: int64

Test de l'api sur une adresse

In [ ]:
location = geocoder.geocode('LYON 6EME')
print(location.latitude, location.longitude)

45.7728396 4.8441256


concaténation de la colonne "address" et "city" et création d'une colonne adresse complémentaire

In [ ]:
df_adress['address_comp'] = df_adress[['address', 'city']].apply(', '.join, axis=1)
df_adress.head()

,address_id,address,city,postal_code,address_comp
0,1,318 CHE DE ROUMAGOUA,LA CIOTAT,13600,"318 CHE DE ROUMAGOUA, LA CIOTAT"
1,2,19 RUE DES DAMES,SAINTE SAVINE,10300,"19 RUE DES DAMES, SAINTE SAVINE"
2,3,22 RUE AMIRAL GUEPRATTE,LE CONQUET,29217,"22 RUE AMIRAL GUEPRATTE, LE CONQUET"
3,4,6 BD DES ETINES,LE COTEAU,42120,"6 BD DES ETINES, LE COTEAU"
4,5,35 AV DU 159EME RIA,BRIANCON,5100,"35 AV DU 159EME RIA, BRIANCON"


In [ ]:
df_adress[['address_comp']].iloc[-1]

address_comp    4 RUE DENIS PAPIN, BLAIN
Name: 561, dtype: object

recherche des coordonées gps

In [ ]:
df_adress['location'] = df_adress['address_comp'].apply(lambda x:geocoder.geocode(x,timeout = 100))

In [ ]:
df_adress['longitude'] = df_adress['location'].apply(lambda x:x.longitude if x else None) 
df_adress['latitude'] = df_adress['location'].apply(lambda x:x.latitude if x else None) 

In [ ]:
df_adress.head(10)
df_adress['address_id'].sum()

171011

recherche du nombre d'adresse sans coordonées gps

In [ ]:
df_adress['location'].isnull().sum()

89

attribution du tableau d'adresse n'ayant pas de coordonées dans une nouvelle variable

In [ ]:
df_nan = df_adress[df_adress['location'].isnull()]
df_nan.head()

,address_id,address,city,postal_code,address_comp,location,longitude,latitude
4,5,35 AV DU 159EME RIA,BRIANCON,5100,"35 AV DU 159EME RIA, BRIANCON",None,NaN,NaN
8,9,35 RUE PORTE ST JEAN,ORLEANS,45000,"35 RUE PORTE ST JEAN, ORLEANS",None,NaN,NaN
13,14,117 RUE DES CHARMETTES,LYON 6EME,69006,"117 RUE DES CHARMETTES, LYON 6EME",None,NaN,NaN
16,17,5 PL MARCEL PAGNOL,SAINT-NAZAIRE,44600,"5 PL MARCEL PAGNOL, SAINT-NAZAIRE",None,NaN,NaN
21,23,20 BD PDT FRANKLIN ROOSEVELT,BORDEAUX,33800,"20 BD PDT FRANKLIN ROOSEVELT, BORDEAUX",None,NaN,NaN


recherche des coordonées de df_nan à l'aide de la ville

In [ ]:
df_nan['location'] = df_nan['city'].apply(lambda x:geocoder.geocode(x,timeout = 10000))
df_nan['longitude'] = df_nan['location'].apply(lambda x:x.longitude if x else None) 
df_nan['latitude'] = df_nan['location'].apply(lambda x:x.latitude if x else None)

<ipython-input-12-a0a846d2080e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nan['location'] = df_nan['city'].apply(lambda x:geocoder.geocode(x,timeout = 10000))
<ipython-input-12-a0a846d2080e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nan['longitude'] = df_nan['location'].apply(lambda x:x.longitude if x else None)
<ipython-input-12-a0a846d2080e>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value ins

ajout des valeurs de la table df_nan à la table df_address

In [ ]:
df_adress.append(df_nan)
df_nan.head(20)

,address_id,address,city,postal_code,address_comp,location,longitude,latitude
4,5,35 AV DU 159EME RIA,BRIANCON,5100,"35 AV DU 159EME RIA, BRIANCON","(Briançon, Hautes-Alpes, Provence-Alpes-Côte d...",6.643631,44.898404
8,9,35 RUE PORTE ST JEAN,ORLEANS,45000,"35 RUE PORTE ST JEAN, ORLEANS","(Orléans, Loiret, Centre-Val de Loire, France ...",1.908607,47.902734
13,14,117 RUE DES CHARMETTES,LYON 6EME,69006,"117 RUE DES CHARMETTES, LYON 6EME","(Le Comptoir du 6ème, Rue Vendôme, Lyon 6e Arr...",4.844126,45.772840
16,17,5 PL MARCEL PAGNOL,SAINT-NAZAIRE,44600,"5 PL MARCEL PAGNOL, SAINT-NAZAIRE","(Saint-Nazaire, Loire-Atlantique, Pays de la L...",-2.213891,47.273352
21,23,20 BD PDT FRANKLIN ROOSEVELT,BORDEAUX,33800,"20 BD PDT FRANKLIN ROOSEVELT, BORDEAUX","(Bordeaux, Gironde, Nouvelle-Aquitaine, France...",-0.580036,44.841225
26,29,24 AV DE LA SERANE,MARSEILLE 8EME,13008,"24 AV DE LA SERANE, MARSEILLE 8EME","(Dae de Mairie 8eme Groupe, 246, Rue de Lyon, ...",5.364699,43.333141
31,34,7 RES DE LA VIERGE,EYGUIERES,13430,"7 RES DE LA VIERGE, EYGUIERES","(Eyguières, Aix-en-Provence, Bouches-du-Rhône,...",5.029721,43.695131
33,37,28 LOT MONT VERNON I,SAINT MARTIN,97150,"28 LOT MONT VERNON I, SAINT MARTIN","(Saint-Martin (France), 97150, France, (18.081...",-63.046713,18.081407
48,53,1 ALL DES ANC COMBATTANTS,ASNIERES-SUR-SEINE,92600,"1 ALL DES ANC COMBATTANTS, ASNIERES-SUR-SEINE","(Asnières-sur-Seine, Arrondissement de Nanterr...",2.289045,48.910595
49,54,105 RUE DE L OURCQ,PARIS 19,75019,"105 RUE DE L OURCQ, PARIS 19","(Paris 19, Khu đô thị Vinhomes Imperia, Phường...",106.667553,20.865531


In [ ]:
df_adress = df_adress.append(df_nan, ignore_index=True)
df_nan.head(30)

,address_id,address,city,postal_code,address_comp,location,longitude,latitude
4,5,35 AV DU 159EME RIA,BRIANCON,5100,"35 AV DU 159EME RIA, BRIANCON","(Briançon, Hautes-Alpes, Provence-Alpes-Côte d...",6.643631,44.898404
8,9,35 RUE PORTE ST JEAN,ORLEANS,45000,"35 RUE PORTE ST JEAN, ORLEANS","(Orléans, Loiret, Centre-Val de Loire, France ...",1.908607,47.902734
13,14,117 RUE DES CHARMETTES,LYON 6EME,69006,"117 RUE DES CHARMETTES, LYON 6EME","(Le Comptoir du 6ème, Rue Vendôme, Lyon 6e Arr...",4.844126,45.772840
16,17,5 PL MARCEL PAGNOL,SAINT-NAZAIRE,44600,"5 PL MARCEL PAGNOL, SAINT-NAZAIRE","(Saint-Nazaire, Loire-Atlantique, Pays de la L...",-2.213891,47.273352
21,23,20 BD PDT FRANKLIN ROOSEVELT,BORDEAUX,33800,"20 BD PDT FRANKLIN ROOSEVELT, BORDEAUX","(Bordeaux, Gironde, Nouvelle-Aquitaine, France...",-0.580036,44.841225
26,29,24 AV DE LA SERANE,MARSEILLE 8EME,13008,"24 AV DE LA SERANE, MARSEILLE 8EME","(Dae de Mairie 8eme Groupe, 246, Rue de Lyon, ...",5.364699,43.333141
31,34,7 RES DE LA VIERGE,EYGUIERES,13430,"7 RES DE LA VIERGE, EYGUIERES","(Eyguières, Aix-en-Provence, Bouches-du-Rhône,...",5.029721,43.695131
33,37,28 LOT MONT VERNON I,SAINT MARTIN,97150,"28 LOT MONT VERNON I, SAINT MARTIN","(Saint-Martin (France), 97150, France, (18.081...",-63.046713,18.081407
48,53,1 ALL DES ANC COMBATTANTS,ASNIERES-SUR-SEINE,92600,"1 ALL DES ANC COMBATTANTS, ASNIERES-SUR-SEINE","(Asnières-sur-Seine, Arrondissement de Nanterr...",2.289045,48.910595
49,54,105 RUE DE L OURCQ,PARIS 19,75019,"105 RUE DE L OURCQ, PARIS 19","(Paris 19, Khu đô thị Vinhomes Imperia, Phường...",106.667553,20.865531


Tri des adresses par id

In [ ]:
df_adress = df_adress.sort_values(by=['address_id'])
df_adress.head(30)

,address_id,address,city,postal_code,address_comp,location,longitude,latitude
0,1,318 CHE DE ROUMAGOUA,LA CIOTAT,13600,"318 CHE DE ROUMAGOUA, LA CIOTAT","(Chemin de Roumagoua, La Fieloula, La Ciotat, ...",5.611266,43.195126
1,2,19 RUE DES DAMES,SAINTE SAVINE,10300,"19 RUE DES DAMES, SAINTE SAVINE","(Rue des Dames, Sainte-Savine, Troyes, Aube, G...",4.045789,48.290905
2,3,22 RUE AMIRAL GUEPRATTE,LE CONQUET,29217,"22 RUE AMIRAL GUEPRATTE, LE CONQUET","(Rue Amiral Guépratte, Le Croaë, Le Conquet, B...",-4.768694,48.361279
3,4,6 BD DES ETINES,LE COTEAU,42120,"6 BD DES ETINES, LE COTEAU","(Boulevard des Etines, Le Coteau, Roanne, Loir...",4.092120,46.021659
562,5,35 AV DU 159EME RIA,BRIANCON,5100,"35 AV DU 159EME RIA, BRIANCON","(Briançon, Hautes-Alpes, Provence-Alpes-Côte d...",6.643631,44.898404
4,5,35 AV DU 159EME RIA,BRIANCON,5100,"35 AV DU 159EME RIA, BRIANCON","(Briançon, Hautes-Alpes, Provence-Alpes-Côte d...",6.643631,44.898404
5,6,36 RUE D EN HAUT,BRUNEMONT,59151,"36 RUE D EN HAUT, BRUNEMONT","(Rue d'En Haut, Brunémont, Douai, Nord, Hauts-...",3.141669,50.271954
6,7,1625 RTE DE CHAMBERY,SAINT-ISMIER,38330,"1625 RTE DE CHAMBERY, SAINT-ISMIER","(Saint Ismier, Route de Chambéry, Le Fangeat, ...",5.829954,45.245535
7,8,3 RUE MONTESQUIEU,ASNIERES-SUR-SEINE,92600,"3 RUE MONTESQUIEU, ASNIERES-SUR-SEINE","(Rue Montesquieu, Centre-Ville, Asnières-sur-S...",2.294828,48.911836
563,9,35 RUE PORTE ST JEAN,ORLEANS,45000,"35 RUE PORTE ST JEAN, ORLEANS","(Orléans, Loiret, Centre-Val de Loire, France ...",1.908607,47.902734


suppression des doubons

In [ ]:
#df_adress = df_adress.drop_duplicates(subset=['address_id'], keep='last')
df_adress = df_adress.dropna()
df_adress = df_adress.drop_duplicates(subset=['address_id'], keep='last')
df_adress.head(40)

,address_id,address,city,postal_code,address_comp,location,longitude,latitude
0,1,318 CHE DE ROUMAGOUA,LA CIOTAT,13600,"318 CHE DE ROUMAGOUA, LA CIOTAT","(Chemin de Roumagoua, La Fieloula, La Ciotat, ...",5.611266,43.195126
1,2,19 RUE DES DAMES,SAINTE SAVINE,10300,"19 RUE DES DAMES, SAINTE SAVINE","(Rue des Dames, Sainte-Savine, Troyes, Aube, G...",4.045789,48.290905
2,3,22 RUE AMIRAL GUEPRATTE,LE CONQUET,29217,"22 RUE AMIRAL GUEPRATTE, LE CONQUET","(Rue Amiral Guépratte, Le Croaë, Le Conquet, B...",-4.768694,48.361279
3,4,6 BD DES ETINES,LE COTEAU,42120,"6 BD DES ETINES, LE COTEAU","(Boulevard des Etines, Le Coteau, Roanne, Loir...",4.092120,46.021659
4,5,35 AV DU 159EME RIA,BRIANCON,5100,"35 AV DU 159EME RIA, BRIANCON","(Briançon, Hautes-Alpes, Provence-Alpes-Côte d...",6.643631,44.898404
5,6,36 RUE D EN HAUT,BRUNEMONT,59151,"36 RUE D EN HAUT, BRUNEMONT","(Rue d'En Haut, Brunémont, Douai, Nord, Hauts-...",3.141669,50.271954
6,7,1625 RTE DE CHAMBERY,SAINT-ISMIER,38330,"1625 RTE DE CHAMBERY, SAINT-ISMIER","(Saint Ismier, Route de Chambéry, Le Fangeat, ...",5.829954,45.245535
7,8,3 RUE MONTESQUIEU,ASNIERES-SUR-SEINE,92600,"3 RUE MONTESQUIEU, ASNIERES-SUR-SEINE","(Rue Montesquieu, Centre-Ville, Asnières-sur-S...",2.294828,48.911836
563,9,35 RUE PORTE ST JEAN,ORLEANS,45000,"35 RUE PORTE ST JEAN, ORLEANS","(Orléans, Loiret, Centre-Val de Loire, France ...",1.908607,47.902734
9,10,67 BD COMMANDANT CHARCOT,NEUILLY-SUR-SEINE,92200,"67 BD COMMANDANT CHARCOT, NEUILLY-SUR-SEINE","(67, Boulevard du Commandant Charcot, Neuilly-...",2.258147,48.880572


Suppression des colonnes crées pour réalisé la recherche

In [ ]:
df_adress = df_adress.drop(columns=['address_comp'])
df_adress = df_adress.drop(columns=['location'])
df_adress.head(100)

KeyError: ignored

In [ ]:
df_adress.head(700)

,address_id,address,city,postal_code,longitude,latitude
0,1,318 CHE DE ROUMAGOUA,LA CIOTAT,13600,5.611266,43.195126
1,2,19 RUE DES DAMES,SAINTE SAVINE,10300,4.045789,48.290905
2,3,22 RUE AMIRAL GUEPRATTE,LE CONQUET,29217,-4.768694,48.361279
3,4,6 BD DES ETINES,LE COTEAU,42120,4.092120,46.021659
4,5,35 AV DU 159EME RIA,BRIANCON,5100,6.643631,44.898404
...,...,...,...,...,...,...
557,601,20 RUE COLI,MONTREUIL,93100,2.468415,48.868459
558,602,6 AV JEAN JAURES,FEURS,42110,4.226839,45.743563
559,603,1 RUE HENRI RENAUDIN,CHARLEVILLE-MEZIERES,8000,4.708829,49.773546
560,604,2 RUE DE LYON,EPINAY SUR SEINE,93800,2.292919,48.953836


In [ ]:
df_update = df_adress
df_update.head(100)


KeyError: ignored

création d'un fichier csv avec la nouvelle table

In [ ]:
from google.colab import files
df_update.to_csv('address_update.csv')
files.download('address_update.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>